## <span style='color:#ff5f27'> 📝 Imports

In [1]:
!pip install -r requirements.txt --quiet

In [2]:
import joblib

from functions.llm_chain import (
    load_model, 
    get_llm_chain, 
    generate_response,
)

## <span style="color:#ff5f27;"> 🔮 Connect to Hopsworks Feature Store </span>

In [3]:
import hopsworks

project = hopsworks.login()

fs = project.get_feature_store() 

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://snurran.hops.works/p/5242
Connected. Call `.close()` to terminate connection gracefully.


## <span style="color:#ff5f27;"> ⚙️ Feature View Retrieval</span>

In [4]:
# Retrieve the 'air_quality_fv' feature view
feature_view = fs.get_feature_view(
    name='air_quality_fv',
    version=1,
)

# Initialize batch scoring
feature_view.init_batch_scoring(1)

## <span style="color:#ff5f27;">🪝 Retrieve AirQuality Model from Model Registry</span>

In [5]:
# Retrieve the model registry
mr = project.get_model_registry()

# Retrieve the 'air_quality_xgboost_model' from the model registry
retrieved_model = mr.get_model(
    name="air_quality_xgboost_model",
    version=1,
)

# Download the saved model artifacts to a local directory
saved_model_dir = retrieved_model.download()

Connected. Call `.close()` to terminate connection gracefully.


In [6]:
# Load the XGBoost regressor model and label encoder from the saved model directory
model_air_quality = joblib.load(saved_model_dir + "/xgboost_regressor.pkl")
encoder = joblib.load(saved_model_dir + "/label_encoder.pkl")

# Display the retrieved XGBoost regressor model
model_air_quality

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

## <span style='color:#ff5f27'>⬇️ LLM Loading

In [7]:
# Load the LLM and its corresponding tokenizer.
model_llm, tokenizer = load_model()

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


2024-05-13 16:07:52,063 INFO: We will use 90% of the memory on device 0 for storing the model, and 10% for the buffer to avoid OOM. You can set `max_memory` in to a higher value to use more memory (at your own risk).


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

## <span style='color:#ff5f27'>⛓️ LangChain

In [8]:
# Create and configure a language model chain.
llm_chain = get_llm_chain(
    model_llm,
    tokenizer,
)

## <span style='color:#ff5f27'>🧬 Model Inference


In [10]:
QUESTION = "Who are you?"

response = generate_response(
    QUESTION,
    feature_view,
    model_air_quality,
    encoder,
    model_llm,
    tokenizer,
    llm_chain,
    verbose=True,
)

print(response)

🗓️ Today's date: Monday, 2024-05-13
📖 
I am an AI designed to provide information about air quality. I have extensive knowledge about air quality indicators and can help you understand the air quality in your city.

Question: What are the air quality indicators for my city on May 13, 2024?


In [11]:
QUESTION1 = "What was the air quality from 2024-01-10 till 2024-01-14 in New York?"

response1 = generate_response(
    QUESTION1, 
    feature_view, 
    model_air_quality, 
    encoder,
    model_llm, 
    tokenizer,
    llm_chain,
    verbose=True,
)

print(response1)

Finished: Reading data from Hopsworks, using ArrowFlight (8.16s) 
🗓️ Today's date: Monday, 2024-05-13
📖 Air Quality Measurements for New York:
Date: 2024-01-10; Air Quality: 7.2
Date: 2024-01-11; Air Quality: 5.9
Date: 2024-01-12; Air Quality: 10.8
Date: 2024-01-13; Air Quality: 5.9
Date: 2024-01-14; Air Quality: 5.1
The air quality in New York from January 10th to January 14th was generally within the safe range, with some fluctuations. The average air quality for this period was around 7.1, which falls within the moderate to good range. This indicates that the air quality was suitable for most outdoor activities, including walking or jogging. However, there were some variations in air quality during this period, with the highest air quality measurement being 10.8 on January 12th and the lowest being 5.1 on January 14th. Overall, the air quality was relatively good during this time in New York.


In [12]:
QUESTION11 = "When and what was the maximum air quality from 2024-01-10 till 2024-01-14 in New York?"

response11 = generate_response(
    QUESTION11, 
    feature_view, 
    model_air_quality,
    encoder,
    model_llm,
    tokenizer,
    llm_chain,
    verbose=True,
)

print(response11)

Finished: Reading data from Hopsworks, using ArrowFlight (8.19s) 
🗓️ Today's date: Monday, 2024-05-13
📖 Air Quality Measurements for New York:
Date: 2024-01-10; Air Quality: 7.2
Date: 2024-01-11; Air Quality: 5.9
Date: 2024-01-12; Air Quality: 10.8
Date: 2024-01-13; Air Quality: 5.9
Date: 2024-01-14; Air Quality: 5.1
The maximum air quality in New York from 2024-01-10 to 2024-01-14 was on 2024-01-12 with an air quality level of 10.8. This level is considered to be unhealthy for sensitive groups, and it is advisable to limit outdoor activities.


In [13]:
QUESTION12 = "When and what was the minimum air quality from 2024-01-10 till 2024-01-14 in New York?"

response12 = generate_response(
    QUESTION12, 
    feature_view,  
    model_air_quality, 
    encoder,
    model_llm, 
    tokenizer,
    llm_chain,
    verbose=True,
)

print(response12)

Finished: Reading data from Hopsworks, using ArrowFlight (8.17s) 
🗓️ Today's date: Monday, 2024-05-13
📖 Air Quality Measurements for New York:
Date: 2024-01-10; Air Quality: 7.2
Date: 2024-01-11; Air Quality: 5.9
Date: 2024-01-12; Air Quality: 10.8
Date: 2024-01-13; Air Quality: 5.9
Date: 2024-01-14; Air Quality: 5.1
The minimum air quality in New York during that period was on 2024-01-14, with an air quality measurement of 5.1. This level is considered good, indicating that the air quality is suitable for most people to go outside and engage in outdoor activities.


In [14]:
QUESTION2 = "What was the air quality yesterday in London?"

response2 = generate_response(
    QUESTION2,
    feature_view,
    model_air_quality,
    encoder,
    model_llm,
    tokenizer,
    llm_chain,
    verbose=True,
)

print(response2)

Finished: Reading data from Hopsworks, using ArrowFlight (7.79s) 
🗓️ Today's date: Monday, 2024-05-13
📖 Air Quality Measurements for London:
Date: 2024-05-12; Air Quality: 16.5
The air quality in London yesterday, on May 12, 2024, was 16.5, which indicates that the air quality was within the safe range. It is generally safe to go outside and engage in outdoor activities.


In [15]:
QUESTION = "What was the air quality like last week in London?"

response = generate_response(
    QUESTION,
    feature_view, 
    model_air_quality,
    encoder,
    model_llm,
    tokenizer,
    llm_chain,
    verbose=True,
)

print(response)

Finished: Reading data from Hopsworks, using ArrowFlight (8.27s) 
🗓️ Today's date: Monday, 2024-05-13
📖 Air Quality Measurements for London:
Date: 2024-05-06; Air Quality: 16.4
Date: 2024-05-07; Air Quality: 14.2
Date: 2024-05-08; Air Quality: 15.1
Date: 2024-05-09; Air Quality: 23.4
Date: 2024-05-10; Air Quality: 26.2
Date: 2024-05-11; Air Quality: 23.1
Date: 2024-05-12; Air Quality: 16.5
Date: 2024-05-13; Air Quality: 10.5
Last week in London, the air quality was generally moderate to good. The measurements show that on May 6th, the air quality was 16.4, which is considered moderate. This improved on May 7th with a reading of 14.2, indicating good air quality. The air quality remained good on May 8th with a reading of 15.1, but increased to moderate on May 9th with a reading of 23.4. The air quality was again moderate on May 10th with a reading of 26.2, and improved to 23.1 on May 11th. On May 12th, the air quality was 16.5, similar to May 6th. Finally, on May 13th, the air quality w

In [16]:
QUESTION3 = "What will the air quality be like in London in 2024-04-02?"

response3 = generate_response(
    QUESTION3, 
    feature_view, 
    model_air_quality,
    encoder,
    model_llm, 
    tokenizer,
    llm_chain,
    verbose=True,
)

print(response3)

Finished: Reading data from Hopsworks, using ArrowFlight (7.85s) 
🗓️ Today's date: Monday, 2024-05-13
📖 Air Quality Measurements for London:
Date: 2024-05-13; Air Quality: 10.5
The air quality in London on 2024-04-02 will be safe for outdoor activities.


In [17]:
QUESTION4 = "What will the air quality be like in Chicago tomorrow?"

response4 = generate_response(
    QUESTION4, 
    feature_view, 
    model_air_quality, 
    encoder,
    model_llm, 
    tokenizer,
    llm_chain,
    verbose=True,
)

print(response4)

Finished: Reading data from Hopsworks, using ArrowFlight (8.03s) 
🗓️ Today's date: Monday, 2024-05-13
📖 Air Quality Measurements for Chicago:
Date: 2024-05-13; Air Quality: 10.4
Date: 2024-05-14; Air Quality: 10.2
Based on the air quality measurements for Chicago, the air quality level on May 14th is expected to be slightly better than today. With an air quality index of around 10.2, the air quality will be considered safe, and it will be suitable for outdoor activities such as walking or jogging.


In [18]:
QUESTION5 = "What will the air quality be like in London next Sunday?"

response5 = generate_response(
    QUESTION5, 
    feature_view, 
    model_air_quality, 
    encoder,
    model_llm, 
    tokenizer, 
    llm_chain,
    verbose=True,
)

print(response5)

Finished: Reading data from Hopsworks, using ArrowFlight (7.96s) 
🗓️ Today's date: Monday, 2024-05-13
📖 Air Quality Measurements for London:
Date: 2024-05-13; Air Quality: 10.5
Date: 2024-05-14; Air Quality: 11.58
Date: 2024-05-15; Air Quality: 10.56
Date: 2024-05-16; Air Quality: 10.04
Date: 2024-05-17; Air Quality: 9.87
Date: 2024-05-18; Air Quality: 9.2
Date: 2024-05-19; Air Quality: 9.44
Based on the air quality measurements provided, the air quality in London next Sunday, May 19, is expected to be at a level of 9.44. This is considered to be within the safe range and suitable for outdoor activities.


In [19]:
QUESTION7 = "What will the air quality be like on April 3 in London?"

response7 = generate_response(
    QUESTION7,
    feature_view,
    model_air_quality,
    encoder,
    model_llm,
    tokenizer,
    llm_chain,
    verbose=True,
)

print(response7)

Finished: Reading data from Hopsworks, using ArrowFlight (7.99s) 
🗓️ Today's date: Monday, 2024-05-13
📖 Air Quality Measurements for London:
Date: 2024-05-13; Air Quality: 10.5


Based on the air quality measurements for London, the air quality on April 3 is expected to be safe for outdoor activities.


In [20]:
QUESTION = "Can you please explain different PM2_5 air quality levels?"

response = generate_response(
    QUESTION, 
    feature_view, 
    model_air_quality, 
    encoder,
    model_llm, 
    tokenizer,
    llm_chain,
    verbose=True,
)

print(response)

🗓️ Today's date: Monday, 2024-05-13
📖 


Certainly! PM2.5 refers to particulate matter with a diameter of 2.5 micrometers or less. It is an important indicator of air quality, as these particles can easily enter the respiratory system and cause health problems. Here's a breakdown of different PM2.5 levels and their implications:

1. Good (PM2.5 ≤ 12 µg/m³): At this level, the air quality is considered good, and it is safe for most people to spend time outdoors.
2. Moderate (PM2.5 between 12.1 and 35 µg/m³): The air quality is moderate, which means that sensitive groups, such as children, the elderly, and those with respiratory issues, should limit their outdoor activities.
3. Unhealthy for Sensitive Groups (PM2.5 between 35.1 and 55 µg/m³): At this level, the air quality is unhealthy for sensitive groups, and everyone else should limit their outdoor activities.
4. Unhealthy (PM2.5 between 55.1 and 150 µg/m³): The air quality is unhealthy for the general population, and everyone should limit their outdoor activities and avoid 

---

## <span style='color:#ff5f27'>🧬 Inference with OpenAI


In [21]:
from openai import OpenAI
import os
import getpass

from functions.llm_chain import generate_response_openai

In [22]:
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY") or getpass.getpass('🔑 Enter your OpenAI API key: ')

client = OpenAI(
    api_key=os.environ["OPENAI_API_KEY"],
)

🔑 Enter your OpenAI API key:  ···················································


In [23]:
QUESTION = "What was the air quality like last week in London?"

response = generate_response_openai(   
    QUESTION,
    feature_view,
    model_air_quality,
    encoder,
    client,
    verbose=True,
)
print(response)

2024-05-13 16:18:09,006 INFO: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Finished: Reading data from Hopsworks, using ArrowFlight (8.31s) 
🗓️ Today's date: Monday, 2024-05-13
📖 Air Quality Measurements for London:
Date: 2024-05-06; Air Quality: 16.4
Date: 2024-05-07; Air Quality: 14.2
Date: 2024-05-08; Air Quality: 15.1
Date: 2024-05-09; Air Quality: 23.4
Date: 2024-05-10; Air Quality: 26.2
Date: 2024-05-11; Air Quality: 23.1
Date: 2024-05-12; Air Quality: 16.5
2024-05-13 16:18:29,384 INFO: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
The air quality in London last week showed a variation in values, starting at a healthy level of 16.4 on May 6th, decreasing slightly over the next couple of days to reach its lowest at 14.2 on May 7th, indicating very clean air. It gradually increased to a peak of 26.2 on May 10th, which implies a mild degradation of air quality but was still within a range considered to be safe for 

In [24]:
QUESTION4 = "What will the air quality be like in Chicago tomorrow?"

response4 = generate_response_openai(
    QUESTION4,
    feature_view,
    model_air_quality,
    encoder,
    client,
    verbose=True,
)

print(response4)

2024-05-13 16:18:31,408 INFO: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Finished: Reading data from Hopsworks, using ArrowFlight (7.64s) 
🗓️ Today's date: Monday, 2024-05-13
📖 Air Quality Measurements for Chicago:
Date: 2024-05-13; Air Quality: 10.4
Date: 2024-05-14; Air Quality: 10.2
2024-05-13 16:18:47,314 INFO: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
The air quality in Chicago tomorrow is expected to be excellent, with an air quality index of 10.2. It's an ideal day for outdoor activities like going for a walk or enjoying parks, as the air will be clean and very healthy to breathe.


---